Kaiwen provided the code to convert cine short axis dicom images to mp4 gif files. Mahri compiled all the data together, obtained gif files, and put all the necessary filepaths into scd_patientdata_xlsx excel file

In [ ]:
import os
import pydicom
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
import matplotlib.animation as animation
import pandas as pd

In [ ]:
# User handle where the SCD_Images folders are located
user_handle = r"C:\Users\mkkad\Desktop\github_repos\heart_cardiac_mri_image_processing\data"  

In [ ]:
# Define dataframe of patient data and cine short axis images filepath
patient_data_excel_path = "scd_patientdata_xlsx.xlsx"
patient_data = pd.read_excel(patient_data_excel_path)

# Get the absolute filepath to cine short axis mri images for each patient
abs_filepath = [os.path.join(user_handle, i) for i in patient_data['filepath'].tolist()]

In [ ]:
# Obtain a dictionary where keys are patient filepaths and values of each key are dicom image filepaths
patient_dicom_files = {}

for filepath in abs_filepath:

    dicom_files = [os.path.join(filepath, i) for i in os.listdir(filepath) if i.endswith(".dcm")]

    patient_dicom_files[filepath] = dicom_files

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def create_dicom_gif(images, output_filepath, fps=2):
    if not images:
        print("❌ No DICOM images to process.")
        return

    # Create figure with transparent background
    fig, ax = plt.subplots()
    fig.patch.set_facecolor('none')  # Set the figure background to transparent
    img_display = ax.imshow(images[0], cmap="gray")
    ax.axis("off")

    # Update function for animation
    def update(frame):
        img_display.set_data(images[frame])

        # Explicitly clear previous title
        ax.set_title('')  # Clear the previous title
        # ax.set_title(f"Frame {frame + 1} of {len(images)}", fontsize=12)

        # Adjust layout to ensure title doesn't overlap
        plt.draw()  # Ensure the plot is redrawn
        plt.tight_layout()

    ani = animation.FuncAnimation(
        fig, update, frames=len(images), interval=500  # 500 ms delay = 2 FPS
    )

    # Save the animation as a GIF with a transparent background
    ani.save(output_filepath, writer="pillow", fps=fps)
    plt.close(fig)
    print(f"🎥 gif saved as: {output_filepath}")


In [ ]:
# Generate mp4 gif for every patient and save the gif in their folders
gif_filepaths = []
for idx, (patient, patient_files) in enumerate(patient_dicom_files.items()):


    patient_gif_name = os.path.basename(os.path.dirname(patient)) + "_gif.gif"
    patient_gif_filepath = os.path.join(os.path.dirname(patient), patient_gif_name)
    
    patient_gif_filepath_to_append = os.path.join(os.path.dirname(patient_data['filepath'].tolist()[idx]), patient_gif_name)

    gif_filepaths.append(patient_gif_filepath_to_append)


    patient_dicom_images = []
    for img_path in sorted(patient_files):
        try:
            dicom_data = pydicom.dcmread(img_path)
            patient_dicom_images.append(dicom_data.pixel_array)
        except Exception as e:
            print(f"⚠️ Skipping {img_path} due to error: {e}")
    
    create_dicom_gif(patient_dicom_images, output_filepath=patient_gif_filepath, fps=20)
    

In [ ]:
# Write into dataframe and save it as excel file
patient_data['gif_filepath'] = gif_filepaths
patient_data
patient_data.to_excel(patient_data_excel_path, index=False)

Create a sheet inside the main excel file with patient data called patient_filepaths that will record patient id and dicom image filepaths

In [ ]:
# Obtain all the dicom image filepaths for every patient and save it as a new sheet called "patient_filepaths" inside the excel file called scd_patientdata_xlsx
patient_cropped_dicom_filepaths = []
sheet_name = "patient_filepaths"
patient_id = patient_data['patient_id'].tolist()
cropped_filepaths = patient_data['filepath'].tolist()

for patient, filepath, cropped_filepath in zip(patient_id, abs_filepath, cropped_filepaths):

    dicom_files = [i for i in os.listdir(filepath) if i.endswith(".dcm")]
    dicom_filenames = [ os.path.basename(x) for x in dicom_files]

    dicom_cropped_filepath = [os.path.join(cropped_filepath, y) for y in dicom_filenames]

    # print(f"Patient is {patient} and the dicom images are{dicom_cropped_filepath} \n\n")

    for single_dicom_cropped_filepath in dicom_cropped_filepath:
        patient_cropped_dicom_filepaths.append({"patient_id":patient, "dcm_image_filepath":single_dicom_cropped_filepath})


df = pd.DataFrame(patient_cropped_dicom_filepaths)


try:
    with pd.ExcelWriter(patient_data_excel_path, mode="a", if_sheet_exists="replace") as writer:
        df.to_excel(writer, sheet_name="patient_filepaths", index=False)
except FileNotFoundError:
    with pd.ExcelWriter(patient_data_excel_path, mode="w") as writer:
        df.to_excel(writer, sheet_name="patient_filepaths", index=False)

Create an xlsx file with available masks

In [ ]:
import pandas as pd
from config import *

df = pd.read_excel(patient_data_excel_path, sheet_name=patient_filepaths)
new_df = pd.DataFrame(columns=['patient_id', 'dcm_image_filepath', 'image_filepath', 'mask_filepath', 'csv_filepath'])

for i, row in df.iterrows():
    
    patient_id_row = row["patient_id"]
    patient_dcm_crop_filepath = row['dcm_image_filepath']
    
    patient_dcm_abs_filepath = os.path.join(user_handle, patient_dcm_crop_filepath)

    
    # Finding image path
    patient_img_abs_filepath = patient_dcm_abs_filepath.replace(".dcm", ".png")

    if os.path.exists(patient_img_abs_filepath):
        patient_img_crop_filepath = os.path.relpath(patient_img_abs_filepath, user_handle)
    else:
        patient_img_crop_filepath = None


    # Finding mask path
    patient_mask_abs_filepath = patient_dcm_abs_filepath.replace(".dcm", "_mask.png")

    if os.path.exists(patient_mask_abs_filepath):
        patient_mask_crop_filepath = os.path.relpath(patient_mask_abs_filepath, user_handle)
    else:
        patient_mask_crop_filepath = None

    # Finding csv path
    patient_csv_abs_filepath = patient_dcm_abs_filepath.replace(".dcm", ".csv")

    if os.path.exists(patient_csv_abs_filepath):
        patient_csv_crop_filepath = os.path.relpath(patient_csv_abs_filepath, user_handle)
    
    else:
        patient_csv_crop_filepath = None

    new_df.loc[len(new_df)] = [patient_id_row, patient_dcm_crop_filepath, patient_img_crop_filepath, patient_mask_crop_filepath, patient_csv_crop_filepath]

new_df.to_excel("patient_mask_filepaths.xlsx", sheet_name="mask_filepaths", index=False)

Create a gif with segmentation results

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pydicom
import numpy as np
import cv2
import os

def create_dicom_gif_with_masks(dicom_folder, mask_folder, output_filepath, fps=2):
    # Read DICOM images and masks from respective folders
    dicom_files = [f for f in os.listdir(dicom_folder) if f.endswith('.dcm')]
    mask_files = [f for f in os.listdir(mask_folder) if '_mask_otsu.png' in f]

    dicom_files.sort()  # Sort to ensure the frames are in order
    mask_files.sort()  # Sort to ensure the masks correspond to DICOM images

    if not dicom_files:
        print("❌ No DICOM images found.")
        return

    if not mask_files:
        print("❌ No segmentation masks found.")
        return

    # Create figure with transparent background
    fig, ax = plt.subplots()
    fig.patch.set_facecolor('none')  # Set the figure background to transparent
    ax.axis("off")  # Turn off axis
    
    # Function to overlay mask on DICOM image
    def overlay_mask_on_image(dicom_image, mask_image):
        # Convert DICOM image to 3-channel (RGB) if it is single-channel (grayscale)
        dicom_rgb = np.stack([dicom_image] * 3, axis=-1) if len(dicom_image.shape) == 2 else dicom_image
        
        # Resize the mask to match the DICOM image dimensions
        mask_resized = cv2.resize(mask_image, (dicom_rgb.shape[1], dicom_rgb.shape[0]))

        # Apply the red color (with 50% transparency) to the mask
        mask_overlay = dicom_rgb.copy()
        mask_overlay[mask_resized > 0] = [255, 0, 0]  # Red color for mask area

        # Blend the original image with the mask (50% transparency)
        alpha = 0.5  # Transparency for the mask
        blended_image = cv2.addWeighted(mask_overlay, alpha, dicom_rgb, 1 - alpha, 0)
        
        return blended_image

    # Update function for animation
    def update(frame):
        dicom_file = dicom_files[frame]
        mask_file = mask_files[frame]

        # Load DICOM image
        dicom_image = pydicom.dcmread(os.path.join(dicom_folder, dicom_file)).pixel_array

        # Load the mask image
        mask_image = cv2.imread(os.path.join(mask_folder, mask_file), cv2.IMREAD_GRAYSCALE)

        # Overlay the mask on the DICOM image
        overlapped_image = overlay_mask_on_image(dicom_image, mask_image)

        # Update the image displayed on the plot
        ax.imshow(overlapped_image)
        # ax.set_title(f"Frame {frame + 1} of {len(dicom_files)}", fontsize=12)

        # Explicitly clear previous title
        # ax.set_title(f"Frame {frame + 1} of {len(dicom_files)}")

    # Create animation
    ani = animation.FuncAnimation(
        fig, update, frames=len(dicom_files), interval=500  # 500 ms delay = 2 FPS
    )

    # Save the animation as a GIF
    ani.save(output_filepath, writer="pillow", fps=fps)
    plt.close(fig)
    print(f"🎥 gif saved as: {output_filepath}")

# Example usage
create_dicom_gif_with_masks(
    dicom_folder=r"C:\Users\mkkad\Desktop\github_repos\heart_cardiac_mri_image_processing\data\SCD_IMAGES_01\SCD0000301\CINESAX_5",
    mask_folder=r"C:\Users\mkkad\Desktop\github_repos\heart_cardiac_mri_image_processing\data\SCD_IMAGES_01\SCD0000301\CINESAX_5",
    output_filepath=r"C:\Users\mkkad\Desktop\github_repos\heart_cardiac_mri_image_processing\data\SCD_IMAGES_01\SCD0000301\SCD0000301_segmentation_otsu_gif.gif",
    fps=2
)
